In [ ]:
# Install Tesseract
!apt-get install tesseract-ocr -y

# Install pytesseract and pillow (image processing library)
!pip install pytesseract pillow


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 0s (24.5 MB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 123629 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-e

In [ ]:
import pytesseract
from PIL import Image

# Load the handwritten image
image_path = '/content/image 2.png'
img = Image.open(image_path)

# Use Tesseract to extract text from the image
extracted_text = pytesseract.image_to_string(img)

# Print the extracted text
print("Extracted Text:")
print(extracted_text)

# Save the extracted text to a file
with open('extracted_text 2.txt', 'w') as text_file:
    text_file.write(extracted_text)


Extracted Text:
 

ssentenone (0 08+ YT HLP SOF

Lahore By Ikitchen

Dok Tae,

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

    

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

eo Weekend Breakfast/Lunch ee =
Ailsa) [EG [Sagal Prmloieerooy7#|_ 6pm TOKO)
Pale CA | Sampo OlB2EGNDS AES
Ley TE [primase msm] 62008070 6 | OPM
Dinner
Dac Hosina BFFs
WB [Man2ona [Orryrlyoey? |G aura
ve [basher lOra19246400 | 6 Bi Bopm TF¥©
Con [matshn Cher OP 710-061216 |
TSR Hebi [OLB YOOTT] S|
MD. Sunny

 

 



In [ ]:
import pandas as pd

# 1. Load Zoho CRM data
zoho_data = pd.read_excel('/content/History.xlsx')

# 2. Load IVR call data
ivr_data = pd.read_excel('/content/Call Recording data from Pendulum (April - October 17,2024).xlsx')

# 3. Load Reservation data
reservation_data = pd.read_excel('/content/reservation list update.xlsx')

# 4. Clean and normalize phone numbers for matching
def clean_phone(phone):
    return str(phone).replace(' ', '').replace('-', '')

zoho_data['Phone Number'] = zoho_data['Phone Number'].apply(clean_phone)
ivr_data['Mobile'] = ivr_data['Mobile'].apply(clean_phone)
reservation_data['Mobile Number'] = reservation_data['Mobile Number'].apply(clean_phone)

# 5. Merge datasets based on phone numbers (Zoho, IVR, and Reservation)
merged_data = pd.merge(zoho_data, ivr_data, left_on='Phone Number', right_on='Mobile', how='outer')
merged_data = pd.merge(merged_data, reservation_data, left_on='Phone Number', right_on='Mobile Number', how='outer')

# 6. Group and calculate metrics (e.g., number of visits, interactions)
merged_data['Total_Interactions'] = merged_data.groupby('Phone Number')['Phone Number'].transform('count')

# 7. Sort customers by interactions or sales value (Top 500)
top_500_customers = merged_data.sort_values(by='Total_Interactions', ascending=False).head(500)

# 8. Export the result to Excel
top_500_customers.to_excel('top_500_customers.xlsx', index=False)

print("Top 500 customers list generated successfully!")


/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Top 500 customers list generated successfully!


In [ ]:
import pandas as pd

# Load and clean the datasets
zoho_data = pd.read_excel('/content/History.xlsx')
ivr_data = pd.read_excel('/content/Call Recording data from Pendulum (April - October 17,2024).xlsx')
reservation_data = pd.read_excel('/content/reservation list update.xlsx')
feedback_data = pd.read_excel('/content/Feedback Forms Data.xlsx')

# Standardize phone numbers for consistency
def clean_phone(phone):
    return str(phone).replace(' ', '').replace('-', '')

zoho_data['Phone Number'] = zoho_data['Phone Number'].apply(clean_phone)
ivr_data['Mobile'] = ivr_data['Mobile'].apply(clean_phone)
reservation_data['Mobile Number'] = reservation_data['Mobile Number'].apply(clean_phone)
feedback_data['Phone Number'] = feedback_data['Phone Number'].apply(clean_phone)

# Merge datasets on phone numbers
merged_data = pd.merge(zoho_data, ivr_data, left_on='Phone Number', right_on='Mobile', how='outer', suffixes=('_zoho', '_ivr'))
merged_data = pd.merge(merged_data, reservation_data, left_on='Phone Number', right_on='Mobile Number', how='outer', suffixes=('', '_reservation')) # Assuming reservation_data has unique columns
merged_data = pd.merge(merged_data, feedback_data, on='Phone Number', how='outer', suffixes=('', '_feedback'))  # Specify suffixes to handle common columns

# Convert feedback columns to numeric, handling errors
for col in ['Food Review', 'Service', 'Cleanliness', 'Atmosphere', 'Overall Experience']:
    merged_data[col] = pd.to_numeric(merged_data[col], errors='coerce')

# Fill missing values with 0 to avoid errors in calculations
merged_data.fillna(0, inplace=True)

# Calculate interaction count and feedback score
merged_data['Total_Interactions'] = merged_data.groupby('Phone Number')['Phone Number'].transform('count')
merged_data['Average_Feedback_Score'] = merged_data[['Food Review', 'Service', 'Cleanliness',
                                                     'Atmosphere', 'Overall Experience']].mean(axis=1)

# Convert 'Chat Duration' to minutes for numerical calculation
def duration_to_minutes(duration):
    if isinstance(duration, str):
        parts = duration.split()
        minutes = 0
        for i in range(0, len(parts), 2):
            if parts[i + 1].lower().startswith('day'):
                minutes += int(parts[i]) * 24 * 60
            elif parts[i + 1].lower().startswith('min'):
                minutes += int(parts[i])
        return minutes
    else:
        return duration  # Handle cases where it's already numeric or NaN

# Assign a weighted score to rank customers
merged_data['Final_Score'] = (
    merged_data['Total_Interactions'] * 0.4 +  # 40% weight for interactions
    merged_data['Overall Experience'] * 0.2 +  # 20% weight for feedback score
    # merged_data['Chat Duration'].astype(float) * 0.4  # 40% weight for chat duration (from Zoho CRM)
    merged_data['Chat Duration'].apply(duration_to_minutes)

)

# Sort by Final_Score and select the top 500 customers
top_500_customers = merged_data.sort_values(by='Final_Score', ascending=False).head(500)

# Export the top 500 customers to Excel
top_500_customers.to_excel('top_500_customers_new.xlsx', index=False)

print("Top 500 customers list generated successfully!")


/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
<ipython-input-6-51b84bc5d99f>:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  merged_data.fillna(0, inplace=True)


Top 500 customers list generated successfully!


In [ ]:
import requests

PAT = 'patrg15rI5Et2DNrz.b0ad4007d76a339b8ff23254e0a47c425b7bfffb918ae77882b9ac22b399d787'
BASE_ID = 'appM8Ih1UVEgdTVNc'
TABLE_NAME = 'WhatsApp Conversation'

url = f'https://api.airtable.com/v0/{BASE_ID}/{TABLE_NAME}'
headers = {
    'Authorization': f'Bearer {PAT}',
    'Content-Type': 'application/json'
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    print("Data fetched successfully:", response.json())
else:
    print(f"Failed to fetch data: {response.status_code}")


Data fetched successfully: {'records': [{'id': 'rec01yUwFCMrSbR7u', 'createdTime': '2024-08-01T13:43:57.000Z', 'fields': {'Name/Phone Number': ' +880 1303-233194', 'Category': ['Menu'], 'Support Response': ['Good'], 'Conversation': 'Customer: Hello! Can I get your menu\nLahore by IKitchen: Thank you for contacting Lahore by IKitchen! Please let us know how may we serve you.\nCustomer: Can you please provide your menu\nLahore by IKitchen: IMG-20240520-WA0022.jpg (file attached)\nHere is our Regular Menu for more details-\nCustomer: Thank you'}}, {'id': 'rec06gEksW5C5Hxj5', 'createdTime': '2024-08-01T13:43:57.000Z', 'fields': {'Name/Phone Number': ' Reza Rabbi', 'Category': ['Menu'], 'Support Response': ['Good'], 'Conversation': 'Customer is a contact\nCustomer: Hello! Can I get more info on this?\nLahore by IKitchen: Thank you for contacting Lahore by IKitchen! Please let us know how may we serve you.\nCustomer: Menu please\nLahore by IKitchen: IMG-20240520-WA0022.jpg (file attached)\nH

In [ ]:
import pandas as pd
import requests

if response.status_code == 200:
    whatsapp_data = pd.json_normalize(response.json()['records'])
    print("WhatsApp data fetched successfully!")
else:
    print("Failed to fetch data:", response.status_code)

whatsapp_data.head()

WhatsApp data fetched successfully!


,id,createdTime,fields.Name/Phone Number,fields.Category,fields.Support Response,fields.Conversation
0,rec01yUwFCMrSbR7u,2024-08-01T13:43:57.000Z,+880 1303-233194,[Menu],[Good],Customer: Hello! Can I get your menu\nLahore b...
1,rec06gEksW5C5Hxj5,2024-08-01T13:43:57.000Z,Reza Rabbi,[Menu],[Good],Customer is a contact\nCustomer: Hello! Can I ...
2,rec0DTWax2oLWK8YS,2024-08-01T13:43:57.000Z,+880 1535-803791,[Incomplete Query],[Good],Customer: Hello! Can I get more info on this?\...
3,rec0DnxulKSLlj7ax,2024-08-01T13:43:57.000Z,+880 1716-132288,[Menu],[Good],Customer: Hello! Can I get more info on this?\...
4,rec0IE3x9CQlRpEq5,2024-08-01T13:43:57.000Z,SETU,[Opening Hours],[Good],Customer is a contact\nCustomer: Link:\nhttps:...


In [ ]:
# Group by customer phone number and count interactions
whatsapp_data['Interaction_Count'] = whatsapp_data.groupby('fields.Name/Phone Number')['fields.Name/Phone Number'].transform('count')

# Drop duplicate entries after counting
interactions = whatsapp_data[['fields.Name/Phone Number', 'Interaction_Count']].drop_duplicates()

# Display top customers based on interaction count
interactions = interactions.sort_values(by='Interaction_Count', ascending=False)
print(interactions.head(10))


         fields.Name/Phone Number  Interaction_Count
0                +880 1303-233194                  1
63                           Arko                  1
73               +880 1752-499828                  1
72               +880 1711-988716                  1
71                          Riyad                  1
70                  Sabbir Ahamed                  1
69                 Tanzina Rahman                  1
68                          Rifad                  1
67   Jerin Ma'am HRAdmin Ikitchen                  1
66               +880 1714-209967                  1


In [ ]:
from textblob import TextBlob

# Apply sentiment analysis on each chat message
whatsapp_data['Sentiment_Score'] = whatsapp_data['fields.Conversation'].apply(lambda x: TextBlob(x).sentiment.polarity)

# Calculate average sentiment score per customer
whatsapp_data['Average_Sentiment'] = whatsapp_data.groupby('fields.Name/Phone Number')['Sentiment_Score'].transform('mean')

# Keep only relevant columns
sentiment_data = whatsapp_data[['fields.Name/Phone Number', 'Average_Sentiment']].drop_duplicates()

print(sentiment_data.head(10))


  fields.Name/Phone Number  Average_Sentiment
0         +880 1303-233194           0.250000
1               Reza Rabbi           0.375000
2         +880 1535-803791           0.333333
3         +880 1716-132288           0.022619
4                     SETU           0.400000
5             Raqib Arefin           0.625000
6         +880 1715-220474           0.475000
7         +880 1871-370557           0.625000
8         +880 1711-358193           0.562500
9          Abdullah Rashed           0.340000


In [ ]:
import re

# Regex pattern to match only relevant billing messages
def extract_valid_bills(conversation):
    # Look for messages like "Sir/Ma'am, your total bill is BDT [amount]"
    pattern = r'(?:Sir|Ma\'am).*?your total bill is BDT (\d+)'
    matches = re.findall(pattern, conversation, re.IGNORECASE)
    return list(map(int, matches))  # Convert strings to integers

# Apply the function to extract valid bill amounts
whatsapp_data['Bill Amounts'] = whatsapp_data['fields.Conversation'].apply(extract_valid_bills)

# Calculate total spent by summing all valid bill amounts
whatsapp_data['Total_Spent'] = whatsapp_data['Bill Amounts'].apply(sum)

# Count the number of interactions (i.e., each time a valid bill was paid)
whatsapp_data['Interaction_Count'] = whatsapp_data['Bill Amounts'].apply(len)

print(whatsapp_data)

                   id               createdTime fields.Name/Phone Number  \
0   rec01yUwFCMrSbR7u  2024-08-01T13:43:57.000Z         +880 1303-233194   
1   rec06gEksW5C5Hxj5  2024-08-01T13:43:57.000Z               Reza Rabbi   
2   rec0DTWax2oLWK8YS  2024-08-01T13:43:57.000Z         +880 1535-803791   
3   rec0DnxulKSLlj7ax  2024-08-01T13:43:57.000Z         +880 1716-132288   
4   rec0IE3x9CQlRpEq5  2024-08-01T13:43:57.000Z                     SETU   
..                ...                       ...                      ...   
95  recCDL6Dmjfofb8i5  2024-08-01T13:43:57.000Z           Al amin Sorder   
96  recCK3UYIIyTExp6a  2024-08-01T13:43:57.000Z         Md Faruk Hossain   
97  recCNr5yFdhvyWepc  2024-08-01T13:43:57.000Z           Emroj Koli1998   
98  recCW2IJngz86RsBD  2024-07-11T06:04:23.000Z         +880 1752-151539   
99  recCWupWTYbykaZJ3  2024-07-11T06:04:23.000Z         +880 1914-513293   

        fields.Category fields.Support Response  \
0                [Menu]             

In [ ]:
import pandas as pd

# Load the ServQuick sales data (replace 'servquick_sales.xlsx' with your file path)
sales_data = pd.read_excel('/path/to/servquick_sales.xlsx')

# Display the first few rows to understand the data structure
print(sales_data.head())

# Step 1: Remove rows without essential customer info (Phone Number, Name, or Address)
filtered_data = sales_data.dropna(subset=['Customer mobile'])

# Step 2: Aggregate total spending and number of purchases per customer
customer_summary = (
    filtered_data.groupby(['Customer mobile', 'Customer name'], as_index=False)
    .agg(Total_Spent=('Amount', 'sum'), Purchase_Count=('Amount Paid', 'count'))
)

# Step 3: Sort customers by total spending and frequency of purchases
customer_summary = customer_summary.sort_values(by=['Total_Spent', 'Purchase_Count'], ascending=False)

# Step 4: Pick the top 500 customers
top_500_customers = customer_summary.head(500)

# Save the top 500 customers to an Excel file for further analysis
top_500_customers.to_excel('/path/to/top_500_customers.xlsx', index=False)

print("Top 500 customers saved successfully!")


In [ ]:
import pandas as pd

# Step 1: Load the ServQuick sales data Excel file
file_path = '/content/Sales_summ_by_receipt_full_data.xls'
sales_data = pd.read_excel(file_path)

# Step 2: Clean the data to keep only valid customer entries
# Filter out rows where 'Customer mobile' or 'Customer name' are missing
valid_sales = sales_data.dropna(subset=['Customer mobile', 'Customer name'])

# Step 3: Aggregate total amount spent by each customer
# Group by 'Customer mobile' (or name, if phone numbers have duplicates)
customer_spending = valid_sales.groupby(['Customer mobile', 'Customer name']).agg(
    Total_Spent=('Amount', 'sum'),  # Sum of 'Amount' column
    Purchase_Count=('Receipt no', 'count')  # Count of transactions
).reset_index()

# Step 4: Sort customers by total spent to identify the top 500
top_500_customers = customer_spending.sort_values(by='Total_Spent', ascending=False).head(500)

# Step 5: Export the top 500 customers to a new Excel file (optional)
output_path = '/content/Top_500_Customers.xlsx'
top_500_customers.to_excel(output_path, index=False)

print(top_500_customers.head(10))  # View top 10 customers


       Customer mobile        Customer name  Total_Spent  Purchase_Count
1658       01711688888            Imran Sir    130473.00              10
1889       01713046543            BCB Sojib    122336.29               4
188   +880 1777-651000  S.A.Newaz Chowdhury    113450.00               7
1840       01713017038            Ansar Sir     94807.00               8
1581       01711548184       Syed Emdad Ali     93154.15               8
3875       01973192039     Ms Tayeba Rahman     88967.00               1
2955       01769008277         Sowmik Islam     82128.00               1
1659       01711688888         Mr Imran Sir     81282.00              22
3210       01795990002                 Raju     80201.00              14
858        01309001077     Janata Insurance     73603.00               2


In [ ]:
!pip install fuzzywuzzy

In [ ]:
!pip install recordlinkage


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.9/926.9 kB 12.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re

# Path to your large Excel file
file_path = '/content/Sales_summ_by_receipt_full_data.xls'

# Helper function to clean phone numbers
def clean_phone_number(phone):
    phone = str(phone)
    phone = re.sub(r'\D', '', str(phone))  # Keep only digits
    if phone.startswith('880'):
        phone = '+880' + phone[3:]
    elif not phone.startswith('+880'):
        phone = '+880' + phone
    return phone

# Helper function to normalize customer names
def normalize_name(name):
    name = str(name)
    name = re.sub(r'\b(Mr|Mrs|Miss|Sir|Madam)\b\.?', '', str(name), flags=re.IGNORECASE).strip()
    return ' '.join(name.split())  # Remove extra spaces

# Initialize an empty DataFrame to store aggregated results
aggregated_data = pd.DataFrame(columns=['Cleaned_Phone', 'Normalized_Name', 'Total_Spent', 'Purchase_Count'])

# Step 1: Read the Excel file in chunks
chunk_size = 5000  # Adjust based on available memory
xls = pd.ExcelFile(file_path)  # Create an ExcelFile object

# Get the first sheet name
first_sheet_name = xls.sheet_names[0]

# Iterate through chunks of the first sheet only:
for i, chunk in enumerate(xls.parse(sheet_name=first_sheet_name, chunksize=chunk_size)):
    # Skip the first chunk if it's not a DataFrame (e.g., sheet name or metadata)
    # Check if chunk is a DataFrame before processing
    if not isinstance(chunk, pd.DataFrame):
        print(f"Skipping chunk {i} as it's not a DataFrame.")
        continue  # Skip to the next chunk

    try:
        # Step 2: Clean phone numbers and normalize names in the chunk
        # Check if the columns exist before accessing them
        if 'Customer mobile' in chunk.columns and 'Customer name' in chunk.columns:
            chunk['Cleaned_Phone'] = chunk['Customer mobile'].apply(clean_phone_number)
            chunk['Normalized_Name'] = chunk['Customer name'].apply(normalize_name)

            # Step 3: Group by Cleaned_Phone and Normalized_Name and aggregate results
            batch_agg = chunk.groupby(['Cleaned_Phone', 'Normalized_Name']).agg(
                Total_Spent=('Amount', 'sum'),
                Purchase_Count=('Receipt no', 'count')
            ).reset_index()

            # Step 4: Append the batch result to the aggregated DataFrame
            aggregated_data = pd.concat([aggregated_data, batch_agg], ignore_index=True)
        else:
            print(f"Warning: Chunk {i} does not have the expected columns. Skipping this chunk.")
    except KeyError as e:
        print(f"Error processing chunk {i}: {e}")
        print("Chunk columns:", chunk.columns)  # Print the chunk's columns for debugging

# ... (rest of your code remains the same)

# Step 5: Group the aggregated data again to handle overlapping customers across chunks
final_aggregated = aggregated_data.groupby(['Cleaned_Phone', 'Normalized_Name']).agg(
    Total_Spent=('Total_Spent', 'sum'),
    Purchase_Count=('Purchase_Count', 'sum')
).reset_index()

# Step 6: Sort by Total Spent and select the top 500 customers
top_500_customers = final_aggregated.sort_values(by='Total_Spent', ascending=False).head(500)

# Step 7: Export the top 500 customers to an Excel file
output_path = 'top_500_customers_27-10-24'

Skipping chunk 0 as it's not a DataFrame.
Skipping chunk 1 as it's not a DataFrame.
Skipping chunk 2 as it's not a DataFrame.
Skipping chunk 3 as it's not a DataFrame.
Skipping chunk 4 as it's not a DataFrame.
Skipping chunk 5 as it's not a DataFrame.
Skipping chunk 6 as it's not a DataFrame.
Skipping chunk 7 as it's not a DataFrame.
Skipping chunk 8 as it's not a DataFrame.
Skipping chunk 9 as it's not a DataFrame.
Skipping chunk 10 as it's not a DataFrame.
Skipping chunk 11 as it's not a DataFrame.
Skipping chunk 12 as it's not a DataFrame.
Skipping chunk 13 as it's not a DataFrame.
Skipping chunk 14 as it's not a DataFrame.
Skipping chunk 15 as it's not a DataFrame.
Skipping chunk 16 as it's not a DataFrame.
Skipping chunk 17 as it's not a DataFrame.
Skipping chunk 18 as it's not a DataFrame.
Skipping chunk 19 as it's not a DataFrame.
Skipping chunk 20 as it's not a DataFrame.
Skipping chunk 21 as it's not a DataFrame.
Skipping chunk 22 as it's not a DataFrame.
Skipping chunk 23 as 

In [ ]:
import pandas as pd
import re

# Path to your large Excel file
file_path = '/content/Sales_summ_by_receipt_full_data.xls'

# Helper function to clean phone numbers
def clean_phone_number(phone):
    phone = str(phone)
    phone = re.sub(r'\D', '', str(phone))  # Keep only digits
    if phone.startswith('880'):
        phone = '+880' + phone[3:]
    elif not phone.startswith('+880'):
        phone = '+880' + phone
    return phone

# Helper function to normalize customer names
def normalize_name(name):
    name = str(name)
    name = re.sub(r'\b(Mr|Mrs|Miss|Sir|Madam)\b\.?', '', str(name), flags=re.IGNORECASE).strip()
    return ' '.join(name.split())  # Remove extra spaces

# Initialize an empty DataFrame to store aggregated results
aggregated_data = pd.DataFrame(columns=['Cleaned_Phone', 'Normalized_Name', 'Total_Spent', 'Purchase_Count'])

# Step 1: Read the Excel file in chunks
chunk_size = 5000  # Adjust based on available memory
xls = pd.ExcelFile(file_path)  # Create an ExcelFile object

# Get the first sheet name
first_sheet_name = xls.sheet_names[0]

# Iterate through chunks of the first sheet only, dynamically finding the data start:
data_start_found = False
for i, chunk in enumerate(xls.parse(sheet_name=first_sheet_name, chunksize=chunk_size)):
    # Check if chunk is a DataFrame before processing
    if not isinstance(chunk, pd.DataFrame):
        print(f"Skipping chunk {i} as it's not a DataFrame.")
        continue  # Skip to the next chunk

    # Check if the chunk contains the expected columns, indicating the start of data
    if 'Customer mobile' in chunk.columns and 'Customer name' in chunk.columns and 'Amount' in chunk.columns and 'Receipt no' in chunk.columns:
        data_start_found = True  # Mark the data start

    if data_start_found:  # Process only chunks after the data start is found
        try:
            # Step 2: Clean phone numbers and normalize names in the chunk
            chunk['Cleaned_Phone'] = chunk['Customer mobile'].apply(clean_phone_number)
            chunk['Normalized_Name'] = chunk['Customer name'].apply(normalize_name)

            # Step 3: Group by Cleaned_Phone and Normalized_Name and aggregate results
            batch_agg = chunk.groupby(['Cleaned_Phone', 'Normalized_Name']).agg(
                Total_Spent=('Amount', 'sum'),
                Purchase_Count=('Receipt no', 'count')
            ).reset_index()

            # Step 4: Append the batch result to the aggregated DataFrame
            aggregated_data = pd.concat([aggregated_data, batch_agg], ignore_index=True)

        except KeyError as e:
            print(f"Error processing chunk {i}: {e}")
            print("Chunk columns:", chunk.columns)  # Print the chunk's columns for debugging
    else:
        print(f"Skipping chunk {i} before data start.")

# Step 5: Group the aggregated data again to handle overlapping customers across chunks
final_aggregated = aggregated_data.groupby(['Cleaned_Phone', 'Normalized_Name']).agg(
    Total_Spent=('Total_Spent', 'sum'),
    Purchase_Count=('Purchase_Count', 'sum')
).reset_index()

# Step 6: Sort by Total Spent and select the top 500 customers
top_500_customers = final_aggregated.sort_values(by='Total_Spent', ascending=False).head(500)

# Step 7: Export the top 500 customers to an Excel file
output_path = '/top_500_customers_27-10-24.xlsx'  # Replace with your desired output path
top_500_customers.to_excel(output_path, index=False)

Skipping chunk 0 as it's not a DataFrame.
Skipping chunk 1 as it's not a DataFrame.
Skipping chunk 2 as it's not a DataFrame.
Skipping chunk 3 as it's not a DataFrame.
Skipping chunk 4 as it's not a DataFrame.
Skipping chunk 5 as it's not a DataFrame.
Skipping chunk 6 as it's not a DataFrame.
Skipping chunk 7 as it's not a DataFrame.
Skipping chunk 8 as it's not a DataFrame.
Skipping chunk 9 as it's not a DataFrame.
Skipping chunk 10 as it's not a DataFrame.
Skipping chunk 11 as it's not a DataFrame.
Skipping chunk 12 as it's not a DataFrame.
Skipping chunk 13 as it's not a DataFrame.
Skipping chunk 14 as it's not a DataFrame.
Skipping chunk 15 as it's not a DataFrame.
Skipping chunk 16 as it's not a DataFrame.
Skipping chunk 17 as it's not a DataFrame.
Skipping chunk 18 as it's not a DataFrame.
Skipping chunk 19 as it's not a DataFrame.
Skipping chunk 20 as it's not a DataFrame.
Skipping chunk 21 as it's not a DataFrame.
Skipping chunk 22 as it's not a DataFrame.
Skipping chunk 23 as 

In [ ]:
import pandas as pd
import re

# Path to your large Excel file
file_path = '/content/Sales_summ_by_receipt_full_data.xls'

# Helper function to clean phone numbers
def clean_phone_number(phone):
    phone = str(phone)
    phone = re.sub(r'\D', '', str(phone))  # Keep only digits
    if phone.startswith('880'):
        phone = '+880' + phone[3:]
    elif not phone.startswith('+880'):
        phone = '+880' + phone
    return phone

# Helper function to normalize customer names
def normalize_name(name):
    name = str(name)
    name = re.sub(r'\b(Mr|Mrs|Miss|Sir|Madam)\b\.?', '', str(name), flags=re.IGNORECASE).strip()
    return ' '.join(name.split())  # Remove extra spaces


# Read the entire Excel file into a single DataFrame
try:
    sales_data = pd.read_excel(file_path, sheet_name=0, header=None)  # Assuming data starts from the first sheet

    # Find the header row (row containing column names)
    header_row = None
    for index, row in sales_data.iterrows():
        if 'Customer mobile' in row.values and 'Customer name' in row.values and 'Amount' in row.values and 'Receipt no' in row.values:
            header_row = index
            break

    if header_row is not None:
        # Set the header row and drop rows before it
        sales_data.columns = sales_data.iloc[header_row]
        sales_data = sales_data.iloc[header_row + 1:].reset_index(drop=True)

        # Now apply your data cleaning and aggregation logic to the 'sales_data' DataFrame
        sales_data['Cleaned_Phone'] = sales_data['Customer mobile'].apply(clean_phone_number)
        sales_data['Normalized_Name'] = sales_data['Customer name'].apply(normalize_name)

        # Aggregate matched records and sum the total spent per customer
        aggregated_data = sales_data.groupby(['Cleaned_Phone', 'Normalized_Name']).agg(
            Total_Spent=('Amount', 'sum'),
            Purchase_Count=('Receipt no', 'count')
        ).reset_index()

        # Sort by total spent and pick the top 500 customers
        top_500_customers = aggregated_data.sort_values(by='Total_Spent', ascending=False).head(500)

        # Export to Excel (Optional)
        output_path = 'top_500_customers_27-10-24.xlsx'
        top_500_customers.to_excel(output_path, index=False)

        print(top_500_customers.head(10))
    else:
        print("Error: Could not find header row with expected columns.")

except Exception as e:
    print(f"An error occurred: {e}")

        Cleaned_Phone      Normalized_Name   Total_Spent  Purchase_Count
0                +880                  nan  212400921.07           15578
938   +88001711688888                Imran      238526.0              37
1174  +88001713046543            BCB Sojib     122336.29               4
3711   +8801777651000  S.A.Newaz Chowdhury        113450               7
2510  +88001795990002                 Raju        111210              19
1121  +88001713017038                Ansar         94807               8
859   +88001711548184       Syed Emdad Ali      93154.15               8
3188  +88001973192039     Ms Tayeba Rahman         88967               1
2666  +88001819255255                Imran         84627              12
2251  +88001769008277         Sowmik Islam         82128               1
